In [1]:
from keras.datasets import fashion_mnist,mnist
import wandb
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import copy
import argparse
wandb.login()


wandb: Currently logged in as: gsathish333. Use `wandb login --relogin` to force relogin


True

In [2]:
class Activation_Functions:
    def __init__(self,function):
        self.function=function
    def getActivations(self,x):
        if self.function=='relu':
            return np.maximum(0,x)
        elif self.function=='sigmoid':
            return 1.0/(1+np.exp(-x))
        elif self.function=='tanh':
            return np.tanh(x)
    def getDerivatives(self,x):
        if self.function=='relu':
             return 1*(x>0)
        elif self.function=='sigmoid':
            return sigmoid(x)*(1-sigmoid(x))
        elif self.function=='tanh':
            return (1 - (np.tanh(x)**2))         

def relu(x):
    return np.maximum(0,x)

def reluPrime(x):
    return 1*(x>0)

class Optimizers:
    def __init__(self,method):
        self.method=method
    def update(self,obj,lr,momentum=0.9,nesterov_beta=0.9,x_batch=None,y_batch=None,lambda_=0,adam_beta1=0.9,adam_beta2=0.99,rms_beta=0.9):
        if self.method=='sgd':
            for i in range(len(obj.layers)):
                obj.weights[i]=obj.weights[i]-lr*(2*lambda_*obj.weights[i]+obj.gradients_w[i])
                obj.biases[i]=obj.biases[i]-lr*obj.gradients_b[i]
        elif self.method=='momentum':
            init=0
            new_gradients=[None]*len(obj.gradients_w)
            for i in range(len(obj.layers)):
                if obj.prev_gradients_w!=None:
                    new_gradients[i]=(1-momentum)*obj.gradients_w[i]+obj.prev_gradients_w[i]*momentum
                else:
                    new_gradients[i]=lr*obj.gradients_w[i]
                    
                obj.weights[i]=obj.weights[i]-lr*(new_gradients[i]+2*lambda_*obj.weights[i])
            obj.prev_gradients_w=new_gradients
            
            
            pass
        elif self.method=='rmsprop':
            for i in range(len(obj.layers)):
                obj.rms_v[i]=rms_beta*obj.rms_v[i]+(1-rms_beta)*(obj.gradients_w[i]**2)
                
                obj.weights[i]=obj.weights[i]-(lr/(np.sqrt(obj.rms_v[i])+1e-9))*(obj.gradients_w[i])
                 
                obj.biases[i]=obj.biases[i]-lr*obj.gradients_b[i] 
                
        elif self.method=='adam':
            for i in range(len(obj.layers)):
                obj.adam_m[i]=adam_beta1*obj.adam_m[i]+(1-adam_beta1)*obj.gradients_w[i]
                m_hat=obj.adam_m[i]/(1-np.power(adam_beta1,obj.counter))
                
                obj.adam_v[i]=adam_beta2*obj.adam_v[i]+(1-adam_beta2)*obj.gradients_w[i]*obj.gradients_w[i]
                v_hat=obj.adam_v[i]/(1-np.power(adam_beta2,obj.counter))
                obj.weights[i]=obj.weights[i]-(lr/(np.sqrt(v_hat)+1e-9))*m_hat
                obj.biases[i]=obj.biases[i]-lr*obj.gradients_b[i]
        
        elif self.method=='nadam':
             for i in range(len(obj.layers)):
                obj.adam_m[i]=adam_beta1*obj.adam_m[i]+(1-adam_beta1)*obj.gradients_w[i]
                m_hat=obj.adam_m[i]/(1-adam_beta1)
                
                obj.adam_v[i]=adam_beta2*obj.adam_v[i]+(1-adam_beta2)*obj.gradients_w[i]*obj.gradients_w[i]
                v_hat=obj.adam_v[i]/(1-adam_beta2)
                obj.weights[i]=obj.weights[i]-(lr/(np.sqrt(v_hat)+10))*m_hat
                
                obj.weights[i]=obj.weights[i]-(lr*(np.divide((adam_beta1*m_hat)+((1-adam_beta1)/(1-np.power(adam_beta1,obj.counter)))*obj.gradients_w[i],np.sqrt(v_hat)+1e-9)))
                obj.biases[i]=obj.biases[i]-lr*obj.gradients_b[i]
        elif self.method=='nesterov':
            original_weights=obj.weights.copy()
            original_graidents=obj.gradients_w.copy()
            if obj.prev_gradients_w!=None:
                 for i in range(len(obj.layers)):
                        obj.weights[i]=obj.weights[i]-nesterov_beta*obj.gradients_w[i]
            
            layer_outs,inter_values=obj.forward(x_batch)
            loss=cross_entropy_loss(layer_outs[-1],y_batch)
            if np.isnan(loss):
                print("* layer outs"*50)
                print(layer_outs[-1])
                print("* y batch"*50)
                print(y_batch)
#             loss=0
            obj.compute_deltas(layer_outs,inter_values,y_batch)
            obj.find_gradients(x_batch,layer_outs)
            
            for i in range(len(obj.gradients_w)):
                if obj.prev_gradients_w!=None:
                    obj.gradients_w[i]=original_graidents[i]*nesterov_beta+obj.gradients_w[i]
                obj.weights[i]=original_weights[i]-lr*(obj.gradients_w[i]+2*lambda_*obj.weights[i])
            obj.prev_gradients_w=True
            return loss




def sigmoid(x):
    return 1.0/(1.0+1.0*np.exp(-x))
def sigmoidPrime(x):
    return sigmoid(x)*(1-sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanhPrime(x):
    return (1 - (np.tanh(x)**2))

def intialize_weights(method,rows,cols):
    if method=='uniform':
        return np.random.rand(rows,cols)
    elif method=='xavier':
        return np.random.randn(rows,cols)*np.sqrt(2/(rows+cols))
    else:
        return np.random.randn(rows,cols)

def soft_max(x):
    max_=np.max(x,axis=0)
    x=x-max_
    return np.exp(x)/np.sum(np.exp(x),axis=0)

def soft_max_prime(x):
    return soft_max(x)*(1-soft_max(x))
    

class OneHotEncoder:
    def __init__(self,max_):
        self.max_=max_
    def transform(self,x):
        out=np.zeros((len(x),self.max_))
        out[np.arange(len(x)),x]=1
        return out.T

def cross_entropy_loss(y_pre,y):
    y_pre[y_pre<1e-15]=1e-15
    loss=-np.sum(y*np.log(y_pre))
    return loss/float(y_pre.shape[1])

def getSig():
    return lambda x:1.0/(1+np.exp(-x))

def pre_process(x):
    x=x.reshape(-1,784)
    x=x/255
    return x

In [3]:
class NN:
    def __init__(self,input_size,layers,active_func='sigmoid',weights_init='xavier'):
        self.input_size=input_size
        self.layers=layers
        self.nlayers=len(layers)
        self.activations=Activation_Functions(wandb.config.activation)
        self.optimizer=Optimizers(wandb.config.optimizer)
        self.weights=[]
        self.weights.append(intialize_weights(weights_init,layers[0],input_size))
        self.deltas=[None]*self.nlayers
        self.gradients_w=[None]*self.nlayers
        self.gradients_b=[None]*self.nlayers
        self.prev_gradients_w=None
        self.counter=1
        self.biases=[]
        self.batch_size=wandb.config.batch_size
        for i in range(1,len(layers)):
            self.weights.append(intialize_weights(weights_init,layers[i],layers[i-1]))
        for i in range(len(layers)):
            self.biases.append(np.random.randn(layers[i],1))   
        
        if self.optimizer.method in ['adam','nadam']:
            self.adam_m=copy.deepcopy(self.weights)
            self.adam_v=copy.deepcopy(self.adam_m)
            _=[each.fill(0) for each in self.adam_m]
            _=[each.fill(0) for each in self.adam_v]
        if self.optimizer.method=='rmsprop':
            self.rms_v=copy.deepcopy(self.weights)
            _=[each.fill(0) for each in self.rms_v]
        
        
    def forward(self,x):
        layer_outputs=[]
        inter_values=[]
        
        for i in range(len(self.weights)):
            inter_values.append(self.weights[i].dot(x)+self.biases[i])
            if i!= len(self.weights)-1:
                layer_outputs.append(self.activations.getActivations(inter_values[-1]))
                x=layer_outputs[-1]
        layer_outputs.append(soft_max(inter_values[-1]))
        return layer_outputs,inter_values
    def compute_deltas(self,layer_outputs,inter_values,y_one_hot):
#         self.deltas[self.nlayers-1]=(layer_outputs[self.nlayers-1]-y_one_hot)*soft_max_prime(inter_values[self.nlayers-1])
        self.deltas[self.nlayers-1]=(layer_outputs[self.nlayers-1]-y_one_hot)
        
        for i in range(self.nlayers-2,-1,-1):
            self.deltas[i]=np.matmul(self.weights[i+1].T,self.deltas[i+1])*self.activations.getDerivatives(inter_values[i])
        
    def find_gradients(self,x,layer_outs):
        self.gradients_w[0]=(np.matmul(self.deltas[0],x.T)+2*wandb.config.L2_lambda*self.weights[0])/self.batch_size
        for i in range(1,self.nlayers):
            self.gradients_w[i]=(np.dot(self.deltas[i],layer_outs[i-1].T)+2*wandb.config.L2_lambda*self.weights[i])/self.batch_size
        for i in range(self.nlayers):
            self.gradients_b[i]=np.sum(self.deltas[i],keepdims=True)/self.batch_size
             
    def update_weights(self,lr):
        for each in range(self.nlayers):
            self.optimizer.update(self,lr=0.1)
    
            
    def train(self,x,y,x_valid,y_valid,epochs=1):
        lr=wandb.config.learning_rate
        batch_size=wandb.config.batch_size
        train_losses=[]
        valid_losses=[] 
        epochs=wandb.config.epochs
        for epoch in range(xepochs):                
            i=0
            batch_count=0
            loss=0.0
            while i+batch_size < x.shape[1]:
                batch_count+=1
                x_batch=x[:,i:i+batch_size]
                y_batch=y[:,i:i+batch_size]
                i+=batch_size
                if self.optimizer.method!='nesterov':
                    layer_outs,inter_values=self.forward(x_batch)
                    loss+=cross_entropy_loss(layer_outs[-1],y_batch)
                    self.compute_deltas(layer_outs,inter_values,y_batch)
                    self.find_gradients(x_batch,layer_outs)
                    self.optimizer.update(self,lr,0.1,lambda_=0.0005)
                else:
                    loss+=self.optimizer.update(self,lr,0.001,x_batch=x_batch,y_batch=y_batch,lambda_=0.0005)
                self.counter+=1
                    
            train_losses.append(loss/batch_count)
            layer_outs,inter_values=self.forward(x_valid)
            valid_loss=cross_entropy_loss(layer_outs[-1],y_valid)/(y_valid.shape[1])
            valid_losses.append(valid_loss)
            print(f"epoch {epoch+1} : train loss = {train_losses[-1]} valid loss = {valid_loss}")
            
            
        tr_ac=self.calc_accuracy(x,y)
        val_ac=self.calc_accuracy(x_valid,y_valid)
        print("train accuracy ",tr_ac)
        print("valid accuracy ",val_ac);
        wandb.log({'accuracy':tr_ac,'val_accuracy':val_ac})
        return train_losses,valid_losses
    
    def predict_probas(self,x):
        layer_outs,inter_values=self.forward(x)
        return layer_outs[-1]
        
    def calc_accuracy(self,x,y):
        layer_outs,inter_values=self.forward(x)
        pred=np.argmax(layer_outs[-1],axis=0)
        expected=np.argmax(y,axis=0)
        
        return (np.sum(pred==expected)/y.shape[1])*100
        
        

In [4]:
def load_data(dataset='fashion_mnist'):
    if dataset=='fashion_mnist':
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    else:
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train,x_valid=x_train[:int(len(x_train)*0.8)],x_train[int(len(x_train)*0.8):] #splitting train into train and valid
    y_train,y_valid=y_train[:int(len(y_train)*0.8)],y_train[int(len(y_train)*0.8):]

    x_train=pre_process(x_train)
    x_valid=pre_process(x_valid)
    x_test=pre_process(x_test) 
    
    one_hot=OneHotEncoder(10)
    y_train=one_hot.transform(y_train)
    y_valid=one_hot.transform(y_valid)
    y_test=one_hot.transform(y_test)
    
    
    return x_train,y_train,x_valid,y_valid,x_test,y_test
    

x_train,y_train,x_valid,y_valid,x_test,y_test=load_data()

In [5]:
sweep_config = {
    'method': 'grid', #grid, random
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'epochs': {
            'values': [5,10]
        },
        'num_layers':{
            'values':[3,4,5]
        },
         'layer_size':{
            'values':[32,64,128]
        },
        'L2_lambda':{
            'values':[0,0.0005,0.5]
        },
         'learning_rate':{
            'values':[0.1,0.01,0.001]
        },
         'optimizer': {
            'values': ['sgd','momentum','nesterov','rmsprop','adam','nesterov']
        },
        'batch_size': {
            'values': [16,32,64]
        },
         'intialization':{
            'values':['normal','random','xavier']
        },
        'activation': {
            'values': ['sigmoid','tanh','relu']
        },      
        
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='main_sweeps_V3')

Create sweep with ID: 2g44jjcf
Sweep URL: https://wandb.ai/gsathish333/main_sweeps_V3/sweeps/2g44jjcf


In [34]:
def main():
    with wandb.init() as run:
        layer_size=wandb.config.layer_size
        num_layers=wandb.config.num_layers
        layers=[layer_size]*num_layers
        layers.append(10)
        obj=NN(784,layers)
        train_losses,valid_losses=obj.train(x_train.T,y_train,x_valid.T,y_valid)
    
wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: 69uccak8 with config:
wandb: 	L2_lambda: 0
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	intialization: normal
wandb: 	layer_size: 32
wandb: 	learning_rate: 0.1
wandb: 	num_layers: 3
wandb: 	optimizer: sgd


epoch 1 : train loss = 1.5077584181044346 valid loss = 7.341736303439495e-05
epoch 2 : train loss = 0.7658766764343988 valid loss = 5.5418321916522455e-05
epoch 3 : train loss = 0.6378783679551191 valid loss = 4.819583884319766e-05
epoch 4 : train loss = 0.575243291905668 valid loss = 4.489843442166284e-05
epoch 5 : train loss = 0.5411583263082237 valid loss = 4.254730244662275e-05
train accuracy  82.88333333333333
valid accuracy  82.29166666666666


accuracy,▁
val_accuracy,▁
accuracy,82.88333
val_accuracy,82.29167


wandb: Agent Starting Run: pome64lw with config:
wandb: 	L2_lambda: 0
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	intialization: normal
wandb: 	layer_size: 32
wandb: 	learning_rate: 0.1
wandb: 	num_layers: 3
wandb: 	optimizer: momentum


epoch 1 : train loss = 1.496320305675561 valid loss = 7.10149626279799e-05
epoch 2 : train loss = 0.7443355061287537 valid loss = 5.274269954950156e-05
epoch 3 : train loss = 0.6115788588802396 valid loss = 4.6689399939771086e-05
epoch 4 : train loss = 0.555178837684948 valid loss = 4.3076989622156356e-05
epoch 5 : train loss = 0.5149499346095565 valid loss = 4.0304612878735215e-05
train accuracy  84.0625
valid accuracy  83.5


...

wandb: Agent Starting Run: l7hwsu2v with config:
wandb: 	L2_lambda: 0
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	intialization: normal
wandb: 	layer_size: 64
wandb: 	learning_rate: 0.1
wandb: 	num_layers: 3
wandb: 	optimizer: rmsprop


/var/folders/v8/pvrhyj2x1kdb1h8pp09t18b40000gn/T/ipykernel_9102/2839676318.py:8: RuntimeWarning: overflow encountered in exp
  return 1.0/(1+np.exp(-x))
/var/folders/v8/pvrhyj2x1kdb1h8pp09t18b40000gn/T/ipykernel_9102/2839676318.py:105: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+1.0*np.exp(-x))


epoch 1 : train loss = 1.6053848220692186 valid loss = 8.774399069089049e-05
epoch 2 : train loss = 1.1596295113779616 valid loss = 0.00010720344914630494
epoch 3 : train loss = 1.0922701201702403 valid loss = 8.827378324546881e-05
epoch 4 : train loss = 1.1255547362321907 valid loss = 9.13798167793655e-05
epoch 5 : train loss = 1.0800604535886702 valid loss = 9.40264418444188e-05
train accuracy  67.80833333333334
valid accuracy  67.75


accuracy,▁
val_accuracy,▁
accuracy,67.80833
val_accuracy,67.75


wandb: Agent Starting Run: cam20mmq with config:
wandb: 	L2_lambda: 0
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	intialization: normal
wandb: 	layer_size: 64
wandb: 	learning_rate: 0.1
wandb: 	num_layers: 3
wandb: 	optimizer: adam
wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
 parser = argparse.ArgumentParser(description = 'Set Hyper Parameters')


parser.add_argument('-wp'   , '--wandb_project'  , type = str  , metavar = '', help = 'WandB Project Name (Non-Empty String)                             ')
parser.add_argument('-we'   , '--wandb_entity'   , type = str  , metavar = '', help = 'WandB Entity Name (Non-Empty String)                              ')
parser.add_argument('-d'    , '--dataset'        , type = str  , metavar = '', help = 'Dataset : ["fashion_mnist", "mnist"]                              ')
parser.add_argument('-e'    , '--epochs'         , type = int  , metavar = '', help = 'Number of Epochs (Positive Integer)                               ')
parser.add_argument('-b'    , '--batch_size'     , type = int  , metavar = '', help = 'Batch Size (Positive Integer)                                     ')
parser.add_argument('-l'    , '--loss'           , type = str  , metavar = '', help = 'Loss function : ["mean_squared_error", "cross_entropy"]           ')
parser.add_argument('-o'    , '--optimizer'      , type = str  , metavar = '', help = 'Optimizer : ["sgd", "momentum", "nag", "rmsprop", "adam", "nadam"]')
parser.add_argument('-lr'   , '--learning_rate'  , type = float, metavar = '', help = 'Learning Rate (Positive Float)                                    ')
parser.add_argument('-m'    , '--momentum'       , type = float, metavar = '', help = 'Momentum (Positive Float)                                         ')
parser.add_argument('-beta' , '--beta'           , type = float, metavar = '', help = 'Beta (Positive Float)                                             ')
parser.add_argument('-beta1', '--beta1'          , type = float, metavar = '', help = 'Beta1 (Positive Float)                                            ')
parser.add_argument('-beta2', '--beta2'          , type = float, metavar = '', help = 'Beta2 (Positive Float)                                            ')
parser.add_argument('-eps'  , '--epsilon'        , type = float, metavar = '', help = 'Epsilon (Positive Float)                                          ')
parser.add_argument('-w_d'  , '--weight_decay'   , type = float, metavar = '', help = 'Weight Decay (Positive Float)                                     ')
parser.add_argument('-w_i'  , '--weight_init'    , type = str  , metavar = '', help = 'Weight Init : ["random", "Xavier"]                                ')
parser.add_argument('-nhl'  , '--num_layers'     , type = int  , metavar = '', help = 'Number of HIDDEN Layers (Positive Integer)                        ')
parser.add_argument('-sz'   , '--hidden_size'    , type = int  , metavar = '', help = 'Number of Neurons in Hidden Layers (Positive Integer)             ')
parser.add_argument('-a'    , '--activation'     , type = str  , metavar = '', help = 'Activation Function : ["identity", "sigmoid", "tanh", "ReLU"]     ')
parser.add_argument('-of'   , '--output_function', type = str  , metavar = '', help = 'Activation Function : ["identity", "sigmoid", "tanh", "ReLU"]     ')

# Parse the Input Args
args = parser.parse_args()



In [1]:
# plt.figure(figsize=(10,8))
# plt.plot(np.arange(1,len(train_losses)+1),train_losses,label='train loss')
# plt.plot(np.arange(1,len(valid_losses)+1),valid_losses,label='valid loss')

# plt.xlabel("epoch",fontsize=18)
# plt.ylabel("loss",fontsize=18)
# plt.legend(fontsize=18)

# plt.show()

# out=obj.predict_probas(x_train.T)
# pred=np.argmax(out,axis=0)
# print(classification_report(pred,np.argmax(y_train,axis=0)))

# out=obj.predict_probas(x_valid.T)
# pred=np.argmax(out,axis=0)
# print(classification_report(pred,np.argmax(y_valid,axis=0)))

# out=obj.predict_probas(x_test.T)
# pred=np.argmax(out,axis=0)
# print(classification_report(pred,np.argmax(y_test,axis=0)))

In [329]:
mapping={
0:'T-shirt/top',
1:'Trouser',
2:'Pullover',
3:'Dress',
4:'Coat',
5:'Sandal',
6:'Shirt',
7:'Sneaker',
8:'Bag',
9:'Ankle boot'} 

labels=list(mapping.values())

def plot_confusion_matrix(target,pred):
    plt.figure(figsize=(14,8))
    sns.set(font_scale=1.2)
    sns.heatmap(confusion_matrix(target,pred),cmap=plt.cm.Blues,annot_kws={"fontsize":14},xticklabels=labels,
                yticklabels=labels)
    plt.xlabel('Predcited',fontsize=16)
    plt.ylabel('Actual',fontsize=16)
    plt.title('Confusion matrix (test data)',fontsize=20)
    plt.show()

plot_confusion_matrix(np.argmax(y_test,axis=0),pred)